<a href="https://colab.research.google.com/github/JITHIN-ANTONY-JOSEPH/ERP_11358080/blob/main/8_Experiment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Input : Recipe Ingredients
### Model : Word2Vec(Text) + Node2Vec(Graph) + Neural Networks(Model)

### Mounting to connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importing required libraries and datasets

In [ ]:
import pandas as pd
import json
import random
import networkx as nx
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from jellyfish import jaro_winkler_similarity

with open('/content/drive/My Drive/ERP/modified_Processed_Layer1.json', 'r') as file: # Adjust the path as needed , this is the path to my personal Google Drive
    recipe1m_data = [json.loads(line) for line in file]

sampled_data = random.sample(recipe1m_data, int(1 * len(recipe1m_data)))

# Convert to DataFrame
recipe1m_df = pd.DataFrame(sampled_data)
recipe1m_df['ingredients'] = recipe1m_df['processed_ingredients'].apply(lambda x: ' '.join(x))

# Load the substitution pairs
substitution_pairs_df = pd.read_csv('/content/drive/My Drive/ERP/Recipe1MSubs_full.csv') # Adjust the path as needed , this is the path to my personal Google Drive

# Merge the datasets based on recipe_id (substitution_pairs_df) and id (recipe1m_df)
merged_df = pd.merge(recipe1m_df, substitution_pairs_df, left_on='id', right_on='recipe_id')


In [ ]:
# Load the knowledge graph from the GraphML file
G = nx.read_graphml('/content/drive/My Drive/ERP/knowledge_graph.graphml') # Adjust the path as needed , this is the path to my personal Google Drive

# Extract subgraph with only ingredient nodes
ingredient_nodes = [n for n, attr in G.nodes(data=True) if attr['node_type'] == 'ingredient']
G_ingredients = G.subgraph(ingredient_nodes).copy()


### Embeddings Generation

In [ ]:
# Train Food2Vec model
sentences = merged_df['processed_ingredients'].tolist()
food2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Load the saved node2vec model
node2vec_model = KeyedVectors.load_word2vec_format("/content/drive/My Drive/ERP/node2vec.model")

# Create a mapping from node name to node2vec embeddings
node_to_embedding = {str(node): node2vec_model[str(node)] for node in G_ingredients.nodes()}


In [ ]:
# Generate embeddings for each recipe's ingredients using Food2Vec
ingredient_embeddings = {}
for index, row in merged_df.iterrows():
    ingredients = row['processed_ingredients']
    embeddings = [food2vec_model.wv[ingredient] for ingredient in ingredients if ingredient in food2vec_model.wv]
    if embeddings:
        ingredient_embeddings[row['id']] = np.mean(embeddings, axis=0)

# Convert the processed_ingredients column to sets for faster membership checking
merged_df['processed_ingredients_set'] = merged_df['processed_ingredients'].apply(set)

# Initialize the dictionary with zeros
embedding_size = 100  # Assuming Food2Vec embedding size is 100
ingredient_to_text_embeddings = {ingredient: np.zeros(embedding_size) for ingredient in node_to_embedding.keys()}

# Precompute text embeddings
for ingredient in ingredient_to_text_embeddings.keys():
    filtered_df = merged_df[merged_df['processed_ingredients_set'].apply(lambda x: ingredient in x)]
    text_embedding_list = np.array([ingredient_embeddings[row['id']] for _, row in filtered_df.iterrows()])
    if text_embedding_list.size > 0:
        ingredient_to_text_embeddings[ingredient] = np.mean(text_embedding_list, axis=0)

# Drop the temporary column to clean up the dataframe
merged_df.drop(columns=['processed_ingredients_set'], inplace=True)

# Combine text and graph embeddings for each ingredient
combined_embeddings = {ingredient: np.concatenate((node_to_embedding[ingredient], ingredient_to_text_embeddings[ingredient])) for ingredient in node_to_embedding.keys()}


In [ ]:
ingredient_to_text_embeddings

{'1%_fat_buttermilk': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '1%_fat_cottage_cheese': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 '10%_cream': array([0., 0., 0., 0., 0., 0., 0., 0., 0

In [ ]:
# Filter substitution pairs to ensure both ingredients are in the combined embeddings
substitution_pairs_df = substitution_pairs_df[
    substitution_pairs_df['ingredient1'].isin(combined_embeddings.keys()) &
    substitution_pairs_df['ingredient2'].isin(combined_embeddings.keys())
]

# Prepare training data with positive and negative samples
def prepare_training_data(substitution_pairs_df, combined_embeddings, num_negatives=5):
    train_data = []
    positive_pairs = set()  # To store positive pairs for quick lookup

    # Generate positive samples
    for _, row in substitution_pairs_df.iterrows():
        ingredient1 = row['ingredient1']
        ingredient2 = row['ingredient2']
        if ingredient1 in combined_embeddings and ingredient2 in combined_embeddings:
            embedding1 = combined_embeddings[ingredient1]
            embedding2 = combined_embeddings[ingredient2]
            train_data.append((embedding1, embedding2, 1))  # Positive sample
            positive_pairs.add((ingredient1, ingredient2))
            positive_pairs.add((ingredient2, ingredient1))  # Add both directions for lookup

    # Generate negative samples
    all_ingredients = list(combined_embeddings.keys())
    for ingredient1 in combined_embeddings.keys():
        negative_samples = set()
        while len(negative_samples) < num_negatives:  # Generate specified number of negative samples
            ingredient2 = random.choice(all_ingredients)
            if ingredient1 != ingredient2 and (ingredient1, ingredient2) not in positive_pairs and (ingredient2, ingredient1) not in positive_pairs:
                embedding1 = combined_embeddings[ingredient1]
                embedding2 = combined_embeddings[ingredient2]
                train_data.append((embedding1, embedding2, 0))  # Negative sample
                negative_samples.add(ingredient2)  # Track the ingredient2 used for negatives

    return train_data

# Generate training data
train_data = prepare_training_data(substitution_pairs_df, combined_embeddings)


### Defining and running the Neural Network

In [ ]:
# Define the Kepler model
class KeplerModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(KeplerModel, self).__init__()
        self.fc1 = nn.Linear(input_dim * 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 1)

    def forward(self, x1, x2):
        x = torch.cat((x1, x2), dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_dim = 164  # 100 (Food2Vec) + 64 (node2vec)
hidden_dim = 128
model = KeplerModel(input_dim, hidden_dim)
model = model.to(device)

# Convert to tensors
X1 = torch.tensor(np.array([t[0] for t in train_data]), dtype=torch.float).to(device)
X2 = torch.tensor(np.array([t[1] for t in train_data]), dtype=torch.float).to(device)
y = torch.tensor([t[2] for t in train_data], dtype=torch.float).to(device).unsqueeze(1)

# Train the model
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

def train_kepler():
    model.train()
    optimizer.zero_grad()
    outputs = model(X1, X2)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Training loop
for epoch in range(100):  # Number of epochs can be adjusted
    loss = train_kepler()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}')


Epoch 0, Loss: 0.6988
Epoch 10, Loss: 0.6440
Epoch 20, Loss: 0.5934
Epoch 30, Loss: 0.5213
Epoch 40, Loss: 0.4350
Epoch 50, Loss: 0.3585
Epoch 60, Loss: 0.3201
Epoch 70, Loss: 0.2968
Epoch 80, Loss: 0.2743
Epoch 90, Loss: 0.2538


### Evaluation

In [ ]:
# Function to generate predictions
def generate_kepler_predictions(ingredient, model, embeddings):
    model.eval()
    predictions = []
    if ingredient in embeddings:
        candidates = sorted(
            embeddings.keys(),
            key=lambda ing: model(
                torch.tensor(embeddings[ingredient], dtype=torch.float).unsqueeze(0).to(device),
                torch.tensor(embeddings[ing], dtype=torch.float).unsqueeze(0).to(device)
            ).item(),
            reverse=True
        )[:10]
        predictions.append(candidates)
    return predictions

# Calculate metrics
def calculate_metrics(predictions, ground_truths):
    mrr = 0.0
    hit_1 = 0.0
    hit_3 = 0.0
    hit_10 = 0.0
    for pred, gt in zip(predictions, ground_truths):
        for i, candidate in enumerate(pred):
            if gt == candidate:
                rank = i + 1
                mrr += 1.0 / rank
                if rank == 1:
                    hit_1 += 1.0
                if rank <= 3:
                    hit_3 += 1.0
                if rank <= 10:
                    hit_10 += 1.0
                break
    mrr /= len(ground_truths)
    hit_1 /= len(ground_truths)
    hit_3 /= len(ground_truths)
    hit_10 /= len(ground_truths)
    return mrr, hit_1, hit_3, hit_10

# Function to calculate MRR, Hit@1, Hit@3, and Hit@10 with Jaro-Winkler similarity threshold
def calculate_metrics_with_threshold(predictions, ground_truths, embeddings, threshold=0.8):
    mrr = 0.0
    hit_1 = 0.0
    hit_3 = 0.0
    hit_10 = 0.0
    for pred, gt in zip(predictions, ground_truths):
        for i, candidate in enumerate(pred):
            if gt in embeddings and candidate in embeddings:
                sim = jaro_winkler_similarity(gt, candidate)
                if sim >= threshold:
                    rank = i + 1
                    mrr += 1.0 / rank
                    if rank == 1:
                        hit_1 += 1.0
                    if rank <= 3:
                        hit_3 += 1.0
                    if rank <= 10:
                        hit_10 += 1.0
                    break
    mrr /= len(ground_truths)
    hit_1 /= len(ground_truths)
    hit_3 /= len(ground_truths)
    hit_10 /= len(ground_truths)
    return mrr, hit_1, hit_3, hit_10

# Extract ground truths from the validation set
val_ground_truths = substitution_pairs_df['ingredient2'].tolist()

# Generate predictions for the entire validation set
val_predictions = []
for ingredient in substitution_pairs_df['ingredient1']:
    val_predictions.extend(generate_kepler_predictions(ingredient, model, combined_embeddings))

# Calculate metrics for the Kepler model for the entire validation set
mrr, hit_1, hit_3, hit_10 = calculate_metrics(val_predictions, val_ground_truths)
print(f"Kepler Model (Whole Dataset): MRR: {mrr:.4f}, Hit@1: {hit_1:.4f}, Hit@3: {hit_3:.4f}, Hit@10: {hit_10:.4f}")

# Calculate metrics for the Kepler model with Jaro-Winkler similarity threshold for the entire validation set
mrr, hit_1, hit_3, hit_10 = calculate_metrics_with_threshold(val_predictions, val_ground_truths, combined_embeddings)
print(f"Kepler Model with Jaro-Winkler similarity threshold (Whole Dataset): MRR: {mrr:.4f}, Hit@1: {hit_1:.4f}, Hit@3: {hit_3:.4f}, Hit@10: {hit_10:.4f}")


Kepler Model (Whole Dataset): MRR: 0.0173, Hit@1: 0.0073, Hit@3: 0.0204, Hit@10: 0.0484
Kepler Model with Jaro-Winkler similarity threshold (Whole Dataset): MRR: 0.0356, Hit@1: 0.0150, Hit@3: 0.0423, Hit@10: 0.0958
